In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain
!pip install -U langchain-community
!pip install --prefer-binary pathway
# !pip install torch
!pip install sentence_transformers
# !pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers


  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.82-cp310-cp310-linux_x86_64.whl size=2815258 sha256=18f22d685be690eba7ef98b8cf94937697e08efb1ed247286848e3ec6a7aca91
  Stored in directory: /root/.cache/pip/wheels/5f/da/5a/272c969ba31c678e6bee2543f3be4dfa67d0feb36ed6f94c01
Successfully built llama-cpp-python
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install pathway[xpack-llm]
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
# Necessary Pathway imports
import pandas as pd
import pathway as pw
from pathway.stdlib.indexing import default_vector_document_index
from pathway.xpacks.llm import embedders
from pathway.xpacks.llm.question_answering import answer_with_geometric_rag_strategy_from_index
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from pathway.xpacks.llm.vector_store import VectorStoreServer
from langchain_community.vectorstores import PathwayVectorClient


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

ModuleNotFoundError: No module named 'langchain_huggingface'

In [ ]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Data")
data = loader.load()

In [ ]:
print(data)

[Document(metadata={'source': '/content/drive/MyDrive/Data/Identifying_Parking_Lot_Occupancy_with_YOLOv5.pdf', 'page': 0}, page_content="Identifying Parking Lot Occupancy with YOLOv5  \n \nManato Ogawa, Tomer Arnon# and Edward Gruber1# \n \n1Eastchester High School  \n#Advisor  \n \nABSTRACT  \n \nWith the increasing need for efficient parking space management and growing population, the application of artificial \nintelligence (AI) in occupancy detection has become a topic of significant interest. This paper explores the effective-\nness and reliability  of the YOLO (You Only Look Once) object detection algorithm in differentiating between occu-\npied and empty parking spots. Moreover, it analyzes the impact of the number of training epochs on the overall accu-racy of the AI model. The study utilizes the YOLO  algorithm due to its speed and accuracy which makes the training \nprocess highly efficient. A custom dataset of 135 images was created and annotated for training purposes. The 

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

10

In [ ]:
#get the third chunk
text_chunks[2]

Document(metadata={'source': '/content/drive/MyDrive/Data/Identifying_Parking_Lot_Occupancy_with_YOLOv5.pdf', 'page': 2}, page_content='the custom dataset. The YOLO algorithm is a deep learning -based object detection method known for its real -time \ndetection capabilities [8][9][10]. It performs both object localization (identifying the location of one or more objects \nin an image and drawing  a bounding box around it) and classification (naming the bounding boxes) simultaneously. \nYOLO uses Convolutional Neural Networks (CNN) to detect images.  Convolutional Neural Networks (CNN) is a neural network that looks at a whole group of pixels in an image [11][12]. This allows for feature learning where the \nmodel can analyze the image thoroughly allowing it to spot any features that will help to make a prediction. The \ndarknet architecture of YOLOv5 supports various types of layers including CNN and pooling layers. CNNs employ \nconvolutional layers to extract relevant features from t

In [ ]:
# Step: Create Embeddings for each of the Text Chunk using Pathway
class InputSchema(pw.Schema):
    doc: str

# Create DataFrame from text_chunks
text_df = pd.DataFrame({"doc": [chunk.page_content for chunk in text_chunks]})
documents = pw.debug.table_from_pandas(text_df, schema=InputSchema)

In [ ]:
#Step 06:Downlaod the Embeddings

embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

embedder = embedders.SentenceTransformerEmbedder(
    embedding_model, call_kwargs={"show_progress_bar": False}
)
embedding_dimension: int = embedder.get_embedding_dimension()

In [ ]:
print(embedding_dimension)
index = default_vector_document_index(
    documents.doc, documents, embedder=embedder, dimensions=embedding_dimension
)

384


In [ ]:
# #Step 08: Create Embeddings for each of the Text Chunk
# vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/Model/mistral-7b-instruct-v0.1.Q4_K_M (1).gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/Model/mistral-7b-instruct-v0.1.Q4_K_M (1).gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - k

#Using pathway vector store client

In [ ]:
!pip install --prefer-binary pathway
!pip install pathway[xpack-llm]
!pip install -U langchain-huggingface

In [ ]:
# !pip install -U langchain_community
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from pathway.xpacks.llm.vector_store import VectorStoreServer
from langchain_community.vectorstores import PathwayVectorClient

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
data = pw.io.fs.read(
    "/content/drive/MyDrive/Data",
    format="binary",
    mode="streaming",
    with_metadata=True,
)
splitter = CharacterTextSplitter()


In [ ]:
# Replace with your Localtunnel URL
localtunnel_url = "https://curvy-lions-invite.loca.lt"
host = localtunnel_url.replace("http://", "").replace("https://", "").split(":")[0]
port = "80"

server = VectorStoreServer.from_langchain_components(
    data, embedder=embeddings, splitter=splitter
)

server.run_server(host, port=port, with_cache=True, cache_backend=pw.persistence.Backend.filesystem("./Cache"), threaded=True)

<Thread(VectorStoreServer, started 133338277119552)>

In [ ]:
client = PathwayVectorClient(url="https://demo-document-indexing.pathway.stream")

#RAG pipeline in LangChain

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
retriever = client.as_retriever()

template = """
You are smart assistant that helps users with their documents on Google Drive and Sharepoint.
Given a context, respond to the user question.
CONTEXT:
{context}
QUESTION: {question}
YOUR ANSWER:"""
prompt = ChatPromptTemplate.from_template(template)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
chain.invoke("Hi")


llama_print_timings:        load time =    3952.36 ms
llama_print_timings:      sample time =       8.08 ms /    13 runs   (    0.62 ms per token,  1608.11 tokens per second)
llama_print_timings: prompt eval time =  272033.28 ms /   527 tokens (  516.19 ms per token,     1.94 tokens per second)
llama_print_timings:        eval time =    8362.42 ms /    12 runs   (  696.87 ms per token,     1.43 tokens per second)
llama_print_timings:       total time =  280477.44 ms /   539 tokens


' Hello! How can I assist you with your documents today?'

#USING pathway Query table and Query retriever setup

In [ ]:
# Create the query table
query_df = pd.DataFrame({"query": ["What is linear regression model"]})
query = pw.debug.table_from_pandas(query_df)

In [ ]:
# # Step: Define RetrievalQA using Pathway's retriever setup
# result = query.select(
#     question=query.query,
#     result=answer_with_geometric_rag_strategy_from_index(
#         query.query,
#         index,
#         documents.doc,
#         llm,
#         n_starting_documents=2,
#         factor=2,
#         max_iterations=4,
#         strict_prompt=True,
#     ),
# )
result = query.select(
    question=query.query,
    result=pw.apply(
        lambda q: answer_with_geometric_rag_strategy_from_index(
            q,
            index,
            documents.doc,
            llm,
            n_starting_documents=2,
            factor=2,
            max_iterations=4,
            strict_prompt=True,
        ),
        query.query
    )
)

#Interactive Loop for user queries and answer by the LLM

In [ ]:
import sys

while True:
    user_input = input("Input Prompt: ")
    if user_input == 'exit':
        print('Exiting')
        sys.exit()
    if user_input == '':
        continue

    # Update the query table with user input
    query_df = pd.DataFrame({"query": [user_input]})
    query = pw.debug.table_from_pandas(query_df)

    # Execute the query using the new retriever setup
    result = query.select(
        question=query.query,
        result=pw.apply(
            lambda q: answer_with_geometric_rag_strategy_from_index(
                q,
                index,
                documents.doc,
                llm,
                n_starting_documents=2,
                factor=2,
                max_iterations=4,
                strict_prompt=True,
            ),
            query.query
        )
    )

    # Extract and print the result
    print(f"Answer: {result['result']}")
    result1 = pd.DataFrame(result)
    print(result1)


# while True:
#     user_input = input("Input Prompt: ")
#     if user_input == 'exit':
#         print('Exiting')
#         sys.exit()
#     if user_input == '':
#         continue

#     # Update the query table with user input
#     query_df = pd.DataFrame({"query": [user_input]})
#     query = pw.debug.table_from_pandas(query_df)

#     # Execute the query
#     result = query.select(
#         question=query.query,
#         result=answer_with_geometric_rag_strategy_from_index(
#             query.query,
#             index,
#             documents.doc,
#             llm,
#             n_starting_documents=2,
#             factor=2,
#             max_iterations=4,
#             strict_prompt=True,
#         ),
#     )

#     # Extract and print the result
#     print(f"Answer: {result['result']}")


# while True:
#     user_input = input("Input Prompt: ")
#     if user_input == 'exit':
#         print('Exiting')
#         sys.exit()
#     if user_input == '':
#         continue

#     # Update the query table with user input
#     query_df = pd.DataFrame({"query": [user_input]})
#     query = pw.debug.table_from_pandas(query_df)

#     # Execute the query using the new retriever setup
#     result = query.select(
#         question=query.query,
#         result=pw.apply(
#             lambda q: answer_with_geometric_rag_strategy_from_index(
#               query.query,
#               index,
#               documents.doc,
#               llm,
#               n_starting_documents=2,
#               factor=2,
#               max_iterations=4,
#               strict_prompt=True,
#           ),
#           query.query
#       ),
#   )

#     # Collect and print the result
#     pw.debug.compute_and_print(result)
#     # print(f"Answer: {result['result']}")

Input Prompt: hi
Answer: <table1>.result
                   0
0  <table1>.question
1    <table1>.result


KeyboardInterrupt: Interrupted by user

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=index.as_retriever(search_kwargs={"k": 2}))

AttributeError: 'DataIndex' object has no attribute 'as_retriever'

In [ ]:
query = "What is linear regression model"

In [ ]:
qa.run(query)

' The linear regression model is a statistical method for modeling the relationship between a dependent variable and one or more independent variables, commonly referred to as predictors or explanatory variables. In this paper, the linear regression model is used as a starting point to derive estimators for the variance-covariance matrix of the errors in certain structural change models.'

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Llama.generate: prefix-match hit


Answer:  It is a statistical method used to analyze the linear relationship between two or more variables. In this case, it is used to estimate the coefficients of a linear regression model.


Llama.generate: prefix-match hit


Answer:  sandwich provides various functions for estimating the covariance matrix. The most commonly used ones are vcovHAC and vcovHC. These functions can take different weighting schemes, including kernel-based HAC estimation with automatic bandwidth selection based on weightsAndrews andbwAndrews. Other functions like weightsLumley provide different weighting schemes, such as truncated and smoothed weights, which are useful in certain applications. In econometric analyses, these functions are used to compute partial t-tests for assessing the significance of a parameter. The choice of estimator depends on the presence of heteroscedasticity and autocorrelation in the errors.
